In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageDraw, ImageOps
import numpy as np
from tensorflow.keras.models import load_model
import string

# Load your trained CNN model
model = load_model('mnist.h5')

# EMNIST full label map (example for 62 classes)
label_map = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
]

class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🖋️ EMNIST PRO GUI v3 - Auto-center + Dark Mode + Undo")
        self.dark_mode = False

        # Canvas settings
        self.canvas_size = 400
        self.canvas_bg_color = 'white'
        self.canvas_fg_color = 'black'
        self.canvas = tk.Canvas(root, width=self.canvas_size, height=self.canvas_size, bg=self.canvas_bg_color, cursor="cross")
        self.canvas.pack(pady=10)

        # Drawing image
        self.image = Image.new("L", (self.canvas_size, self.canvas_size), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Brush size
        self.brush_size = tk.IntVar(value=8)

        tk.Label(root, text="🖌️ Brush Size", font=("Helvetica", 14)).pack()
        brush_slider = tk.Scale(root, from_=3, to=20, orient='horizontal', variable=self.brush_size)
        brush_slider.pack()

        # Buttons
        btn_frame = tk.Frame(root)
        btn_frame.pack(pady=10)

        tk.Button(btn_frame, text="🎯 Predict", font=("Helvetica", 12), command=self.predict, bg="#4CAF50", fg="white", width=12).pack(side='left', padx=5)
        tk.Button(btn_frame, text="🗑️ Clear", font=("Helvetica", 12), command=self.clear, bg="#f44336", fg="white", width=12).pack(side='left', padx=5)
        tk.Button(btn_frame, text="↩️ Undo", font=("Helvetica", 12), command=self.undo, bg="#FF9800", fg="white", width=12).pack(side='left', padx=5)
        tk.Button(btn_frame, text="💾 Save Image", font=("Helvetica", 12), command=self.save_image, bg="#2196F3", fg="white", width=12).pack(side='left', padx=5)
        tk.Button(btn_frame, text="🌙 Dark Mode", font=("Helvetica", 12), command=self.toggle_dark_mode, bg="#9C27B0", fg="white", width=12).pack(side='left', padx=5)

        # Result
        self.result = tk.Label(root, text="Draw a character and press Predict", font=("Helvetica", 16))
        self.result.pack(pady=10)

        # Bind drawing
        self.canvas.bind('<B1-Motion>', self.draw_lines)

        # For Undo
        self.stroke_history = []

    def draw_lines(self, event):
        x, y = event.x, event.y
        r = self.brush_size.get()
        oval = self.canvas.create_oval(x - r, y - r, x + r, y + r, fill=self.canvas_fg_color, outline=self.canvas_fg_color)
        self.stroke_history.append(oval)
        self.draw.ellipse([x - r, y - r, x + r, y + r], fill=0)

    def clear(self):
        self.canvas.delete('all')
        self.draw.rectangle([0, 0, self.canvas_size, self.canvas_size], fill=255)
        self.stroke_history.clear()
        self.result.config(text="Draw a character and press Predict")

    def undo(self):
        if self.stroke_history:
            last_stroke = self.stroke_history.pop()
            self.canvas.delete(last_stroke)
            self.redraw_image()

    def redraw_image(self):
        # Re-render full image from current canvas
        self.image = Image.new("L", (self.canvas_size, self.canvas_size), color=255)
        self.draw = ImageDraw.Draw(self.image)
        for item in self.stroke_history:
            coords = self.canvas.coords(item)
            self.draw.ellipse(coords, fill=0)

    def predict(self):
        # Auto-center + auto-rescale the image
        img = self.image.resize((28, 28))
        img = ImageOps.invert(img)

        # Find bounding box of non-white pixels
        bbox = ImageOps.invert(img).getbbox()
        if bbox:
            img = img.crop(bbox)

        # Resize keeping aspect ratio
        img = ImageOps.fit(img, (28, 28), method=Image.BICUBIC)

        # Final processing
        img = np.array(img).astype(np.float32) / 255.0
        img = np.transpose(img)  # EMNIST format
        img = img.reshape(1, 28, 28, 1)

        # Prediction
        pred = model.predict(img)
        top_5 = np.argsort(pred[0])[-5:][::-1]

        # Format result
        result_str = "🔍 Top Predictions:\n"
        for i in top_5:
            char = label_map[i] if i < len(label_map) else '?'
            result_str += f"{char} ({pred[0][i]*100:.2f}%)\n"

        self.result.config(text=result_str)

    def save_image(self):
        filename = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
        if filename:
            img_to_save = self.image.resize((28, 28))
            img_to_save = ImageOps.invert(img_to_save)
            img_to_save.save(filename)
            self.result.config(text=f"✅ Image saved:\n{filename}")

    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        if self.dark_mode:
            self.canvas_bg_color = 'black'
            self.canvas_fg_color = 'white'
            self.canvas.configure(bg='black')
            self.result.configure(bg='black', fg='white')
        else:
            self.canvas_bg_color = 'white'
            self.canvas_fg_color = 'black'
            self.canvas.configure(bg='white')
            self.result.configure(bg='white', fg='black')

if __name__ == '__main__':
    root = tk.Tk()
    app = DrawingApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
